# Projet maison n° 3

In [10]:
# imports
import pandas as pd
import glob

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [11]:
# names us
def df_names_us():
    df = pd.concat([pd.read_csv(filename, names=["name", "gender", "births"]).assign(year=filename[11:-4]) for filename in glob.glob("./names/*.txt")])
    df = df[['year', 'name', 'gender', 'births']]
    df.index = pd.RangeIndex(start=0, stop=len(df))
    return df

In [12]:
df_us = df_names_us()
df_us

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
2052776,2021,Zyeire,M,5
2052777,2021,Zyel,M,5
2052778,2021,Zyian,M,5
2052779,2021,Zylar,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [90]:
# names fr
def df_names_fr():
    df = pd.DataFrame()
    
    # Transformation en dataframe du fichier CSV
    df = pd.read_csv("nat2021.csv", sep=";")
    
    # Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
    df.rename(columns = {'sexe':'gender', 'preusuel':'name', 'annais':'year', 'nombre':'births'}, inplace = True)
    df = df[['year', 'name', 'gender', 'births']]
    
    # Mêmes valeurs pour la colonne "gender" + Mêmes dtypes pour les colonnes
    df.gender = df.gender.map({1: 'M', 2: 'F'})
    
    # Seuls les prénoms de 2 caractères et plus sont conservés (df = df.drop([(df["name"].str.len() >= 2)])
    # df = df.drop(df[(df["name"].str.len() <= 1)].index)
    df = df[(df.name.str.len() >= 2)]
    
    # La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
    df.name = df.name.str.title()
    
    # Les lignes avec des données inutilisables doivent être supprimées
    df = df[(df.name != '_Prenoms_Rares')&(df.year != 'XXXX')]
    
    # Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
    df = df.sort_values(by = ['year', 'gender', 'births', 'name'], ascending = [True, True, False, True])
    
    # L'index du DataFrame doit aller de 0 à N-1
    df.index = pd.RangeIndex(start=0, stop=len(df))
    
    return df

In [91]:
df = pd.read_csv("nat2021.csv", sep=";")
df.rename(columns = {'sexe':'gender', 'preusuel':'name', 'annais':'year', 'nombre':'births'}, inplace = True)
df = df[['year', 'name', 'gender', 'births']]
df.gender = df.gender.map({1: 'M', 2: 'F'})
df.births.isna().value_counts()


False    686538
Name: births, dtype: int64

In [92]:
df_fr = df_names_fr()
na_values = df_fr[df_fr.name.isna()]
na_values

,year,name,gender,births


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [120]:
# taux de change
def df_taux_change(devises):
    df = pd.DataFrame([], columns=devises)
     # Transformation en dataframe du fichier CSV
    df1 = pd.read_csv("Webstat_Export.csv", sep=";")
    #df1.set_index('Titre')
    for columns, series in df1.iteritems():
        df1 = df1.rename(columns = {columns : columns[-4:-1]})
    df1 = df1.iloc[5:]
    df1.re = df1.to_datetime(df1.re, format="%d/%m/%Y")
    #df.index = df1["re"]
    #df = df1[devises]
    return df1

In [121]:
df = df_taux_change(["CHF", "GBP", "USD"])
df

AttributeError: 'DataFrame' object has no attribute 'to_datetime'

### Tests

In [389]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [390]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [391]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ERROR
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... FAIL

ERROR: test_df_names_fr (__main__.Session3Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_472/1439472077.py", line 24, in test_df_names_fr
    self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py", line 967, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py", line 1184, in _getitem_axis
    elif com.is_bool_indexer(key):
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/common.py", line 144, in is_bool_indexer
    raise ValueError(na_msg)
ValueError: Cannot mask with non-boolean array containing NA / NaN values

FAIL: test_df_taux_change (__main__.Session3Tests)
--